In [15]:
using Distributions
using DataFrames
using CSV
using Random

In [16]:
# Seed
rng = MersenneTwister(2137)

MersenneTwister(2137)

In [17]:
function calculate_parameters(number_of_persons::Int64, mean_arrival_rate::Float64, mean_service_rate::Float64)
    return calculated_parameters = Dict(
        'n' => number_of_persons,
        "mean_arrival_rate" => mean_arrival_rate,
        "mean_service_rate" => mean_service_rate,
        "mean_interarrival_time" => 1.0 / mean_arrival_rate,
        "mean_service_time" => 1.0 / mean_service_rate
    )
end

calculate_parameters (generic function with 1 method)

In [18]:
function previous_index(time_df, queue)
    column = time_df[:, Symbol(queue)]
    previous_index = findlast(x -> x == 1.0, column)
    return previous_index
end

previous_index (generic function with 1 method)

In [19]:
function time_dataframe(calculated_parameters::Dict, interarrival_times, arrival_times, service_times)
    # Unpacking parameters
    n = calculated_parameters['n']
    
    # Creating DataFrame
    time_df_head =  DataFrame(
        interarrival_time = 0.0,
        service_time = 0.0,
        arrival_time = 0.0,
        service_start_time = 0.0,
        service_end_time = 0.0,
        wait_time = 0.0,
        queue_1 = 1.0,
        queue_2 = 1.0,
        queue_3 = 1.0
        )
    
    time_df_data = DataFrame(
        interarrival_time = interarrival_times,
        service_time = service_times,
        arrival_time = arrival_times,
        service_start_time = zeros(n),
        service_end_time = zeros(n),
        wait_time = zeros(n),
        queue_1 = zeros(n),
        queue_2 = zeros(n),
        queue_3 = zeros(n)
        )
    
    time_df = vcat(DataFrame(time_df_head), DataFrame(time_df_data)) 
    
    # Loop
    for i in 2:n+1
        # Simplifying variables
        previous_in_queue_1 = previous_index(time_df, "queue_1")
        previous_in_queue_2 = previous_index(time_df, "queue_2")
        previous_in_queue_3 = previous_index(time_df, "queue_3")
        
        # Should go 1 -> 2 -> 3
        if max(time_df[i, :arrival_time], time_df[previous_in_queue_1, :service_end_time]) == time_df[i, :arrival_time]
            time_df[i, :service_start_time] = time_df[i, :arrival_time]
            time_df[i, :service_end_time] = time_df[i, :service_start_time] + time_df[i, :service_time]
            time_df[i, :queue_1] = 1.0
        elseif max(time_df[i, :arrival_time], time_df[previous_in_queue_2, :service_end_time]) == time_df[i, :arrival_time]
            time_df[i, :service_start_time] = time_df[i, :arrival_time]
            time_df[i, :service_end_time] = time_df[i, :service_start_time] + time_df[i, :service_time]
            time_df[i, :queue_2] = 1.0
        elseif max(time_df[i, :arrival_time], time_df[previous_in_queue_3, :service_end_time]) == time_df[i, :arrival_time]
                time_df[i, :service_start_time] = time_df[i, :arrival_time]
                time_df[i, :service_end_time] = time_df[i, :service_start_time] + time_df[i, :service_time]
                time_df[i, :queue_3] = 1.0
        elseif min(
                time_df[previous_in_queue_1, :service_end_time], 
                time_df[previous_in_queue_2, :service_end_time], 
                time_df[previous_in_queue_3, :service_end_time]
                ) == time_df[previous_in_queue_1, :service_end_time]
            time_df[i, :service_start_time] = time_df[previous_in_queue_1, :service_end_time]
            time_df[i, :service_end_time] = time_df[i, :service_start_time] + time_df[i, :service_time]
            time_df[i, :queue_1] = 1.0
        elseif min( 
                time_df[previous_in_queue_2, :service_end_time], 
                time_df[previous_in_queue_3, :service_end_time]
                ) == time_df[previous_in_queue_2, :service_end_time]
            time_df[i, :service_start_time] = time_df[previous_in_queue_2, :service_end_time]
            time_df[i, :service_end_time] = time_df[i, :service_start_time] + time_df[i, :service_time]
            time_df[i, :queue_2] = 1.0
        else 
            time_df[i, :service_start_time] = time_df[previous_in_queue_3, :service_end_time]
            time_df[i, :service_end_time] = time_df[i, :service_start_time] + time_df[i, :service_time]
            time_df[i, :queue_3] = 1.0 
        end
    end
    
    time_df[:, :wait_time] = time_df[:, :service_start_time] - time_df[:, :arrival_time]
    
    CSV.write("model_mmc_time_df.csv", time_df)
    return time_df
end

time_dataframe (generic function with 1 method)

In [20]:
function start_simulation(number_of_persons::Int64, mean_arrival_rate::Float64, mean_service_rate::Float64)
    # Calculating parameters
    calculated_parameters = calculate_parameters(number_of_persons, mean_arrival_rate, mean_service_rate)
    
    # Unpacking parameters
    n = calculated_parameters['n']
    mean_interarrival_time = calculated_parameters["mean_interarrival_time"]
    mean_service_time = calculated_parameters["mean_service_time"]
    
    # Calculating distributions
    interarrival_times = rand(rng, Exponential(mean_interarrival_time), n)
    arrival_times = cumsum(interarrival_times)
    service_times = rand(rng, Exponential(mean_service_time), n)
    
    # Calculating DataFrame 
    time_df = time_dataframe(calculated_parameters, interarrival_times, arrival_times, service_times)
    return time_df
end

start_simulation (generic function with 1 method)

In [21]:
# Parameters and Debugging
time_df = start_simulation(100, 50.0, 100.0)

,interarrival_time,service_time,arrival_time,service_start_time,service_end_time
,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0
2,0.0047415,0.00242425,0.0047415,0.0047415,0.00716575
3,0.00189529,0.000738039,0.0066368,0.0066368,0.00737484
4,0.00172711,0.0122298,0.00836391,0.00836391,0.0205937
5,0.0104642,0.0231278,0.0188281,0.0188281,0.0419559
6,0.0204893,0.0200817,0.0393174,0.0393174,0.0593992
7,0.000197814,0.0113624,0.0395153,0.0395153,0.0508777
8,0.0103539,0.0196161,0.0498692,0.0498692,0.0694853
9,0.00488335,0.0385767,0.0547525,0.0547525,0.0933292
